In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import os
import utils.image_utils as ps

## Load test image

In [ ]:
image_dir = 'test_image.png'
image = np.array(Image.open(image_dir))

## Resize image if necessary

In [ ]:
resize = True

scale = 0.5

if resize:
    image = ps.resize_image(image, percent = scale)
    
plt.imshow(image)

## Pad and crop image

In [ ]:
model_input_w = 512
model_input_h = 512

# pad & crop
padded_image = ps.pad(image, model_input_w, model_input_h)
cropped_tiles = ps.crop(padded_image, model_input_w, model_input_h, numpy_output=True)

## Load model

In [ ]:
from tensorflow.keras.models import load_model
from loss import bce_dice_loss

model = load_model('outputs/weights/my_model.h5',
                   custom_objects={ 'bce_dice_loss': bce_dice_loss} # Load BCE Dice Loss from loss.py into model
                  ) 

## Inference

In [ ]:
model_predictions = model.predict(cropped_tiles,verbose=1)

preds = np.squeeze(model_predictions)

threshold = 0.95
preds = preds > threshold

## Reconstruct final predicted image

In [ ]:
# image height and width
img_h, img_w = image.shape[0], image.shape[1]

# reconstructs predicted mask tiles into one single mask image
predicted_image = ps.reconstruct(preds, img_w, img_h, model_input_w, model_input_h, numpy_output = False)

plt.imshow(predicted_image)

## Save image

In [ ]:
# folder for saving predicted mask image by model
image_save_folder = 'outputs/inference outputs/png'

# creates a new folder if it does not exist yet
os.makedirs(image_save_folder, exists_ok = True)

# save predicted mask image in target folder
predicted_image.save(f'{image_save_folder}/predicted_image.png')